<a href="https://colab.research.google.com/github/TAKE-JP-17/Pytorch/blob/main/22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Earth Engine API
!pip install earthengine-api

# Authenticate and initialize GEE
import ee
from google.colab import auth
auth.authenticate_user()

# Initialize Earth Engine
ee.Initialize()

In [ ]:
# Define a region (replace with Tainan City geometry)
geometry = ee.Geometry.Polygon([
  [[120.0093, 22.7373], [120.3001, 22.7373],
   [120.3001, 23.3308], [120.0093, 23.3308],
   [120.0093, 22.7373]]
])

In [ ]:
def calculate_mndwi(image):
    mndwi = image.normalizedDifference(['GREEN', 'SWIR1']).rename('MNDWI')
    return image.addBands(mndwi)

In [ ]:
def get_annual_image(year):
    # Load the Landsat image collection for the specified year
    collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                .filterDate(f'{year}-01-01', f'{year}-12-31') \
                .filterBounds(tainan) \
                .map(calculate_mndwi) \
                .median()

    return collection.clip(tainan).select('MNDWI')

In [ ]:
def create_binary_mndwi(image, threshold=0):
    return image.gt(threshold).rename('binary_MNDWI')

In [ ]:
# Define the range of years
years = list(range(2000, 2023))  # This will give you 23 years (2000 to 2022)

# Initialize an empty list to store the images
images = []

# Loop through each year and get the July image
for year in years:
    july_image = get_july_image(year)
    images.append(july_image)  # Store each image in the list

# Now you have 23 images (one for each year from 2000 to 2022) stored in the 'images' list

for i, year in enumerate(years):
    july_image = images[i]  # Retrieve the image from the list
    task = ee.batch.Export.image.toDrive(**{
        'image': july_image,
        'description': f'July_MNDWI_{year}',
        'scale': 30,
        'region': tainan,
        'maxPixels': 1e9
    })
    task.start()


In [ ]:
def detect_change(mndwi_current, mndwi_next, threshold=0.2):
    # Calculate the difference between consecutive years
    mndwi_diff = mndwi_next.subtract(mndwi_current).abs()

    # Classify change based on the threshold
    land_to_water = mndwi_diff.gt(threshold).And(mndwi_next.lt(mndwi_current))
    water_to_land = mndwi_diff.gt(threshold).And(mndwi_next.gt(mndwi_current))

    # Generate a change map: 1 = land to water (red), -1 = water to land (blue), 0 = no change (black)
    change_map = land_to_water.multiply(1).add(water_to_land.multiply(-1))
    return change_map

In [ ]:
years = list(range(2000, 2023))

# Store the yearly MNDWI maps
mndwi_maps = [get_july_image(year) for year in years]

# Detect changes between consecutive years
change_maps = []
for i in range(len(mndwi_maps) - 1):
    change = detect_change(mndwi_maps[i], mndwi_maps[i+1])
    change_maps.append(change)

In [ ]:
# Example visualization parameters
viz_params = {
    'min': -1,
    'max': 1,
    'palette': ['0000FF', '000000', 'FF0000']  # Blue, Black, Red
}

# Visualize a change map (e.g., between 2000 and 2001)
import folium
map_ = folium.Map(location=[23.0, 120.25], zoom_start=10)
map_.add_ee_layer(change_maps[0], viz_params, 'Change Map 2000-2001')
map_

In [ ]:
export_task = ee.batch.Export.image.toDrive(**{
    'image': change_maps[0],
    'description': 'MNDWI_change_2000_2001',
    'scale': 30,
    'region': tainan,
    'maxPixels': 1e9
})
export_task.start()